In [32]:
import rpy2.robjects as ro
import pandas as pd
import json
import statistics as st

r = ro.r
r['source']('ParseNFLPlayers.R')

parse_nfl_player_r = ro.globalenv['get_player_data']
parse_position_players_r = ro.globalenv['get_position_players']

In [33]:
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

# This is an R "tibble" of the data
# We load this object from the R script we used earlier
player_stats_r = ro.globalenv['player_stats']
team_data_r = ro.globalenv['team_data']
player_stats_kicker_r = ro.globalenv['player_stats_kicker']
roster_r = ro.globalenv['roster']
official_player_stats_r = ro.globalenv['official_player_stats_total']
weekly_stats = ro.globalenv['official_player_stats']

# Convert the tibble to a pandas data frame
with localconverter(ro.default_converter + pandas2ri.converter):
    df_player_stats = ro.conversion.rpy2py(player_stats_r)
    df_team_data = ro.conversion.rpy2py(player_stats_r)
    df_player_stats_kicker = ro.conversion.rpy2py(player_stats_kicker_r)
    df_roster = ro.conversion.rpy2py(roster_r)
    df_official_player_stats = ro.conversion.rpy2py(official_player_stats_r)
    df_weekly_stats = ro.conversion.rpy2py(weekly_stats)

In [102]:
df_official_player_stats.head(20)

,player_id,player_name,games,recent_team,completions,attempts,passing_yards,passing_tds,interceptions,sacks,...,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr,consistency_grade
1,00-0019596,T.Brady,19,TB,544,810,5916.0,46,13.0,29.0,...,NaN,0,NaN,NaN,NaN,NaN,0.0,406.74,406.74,F
2,00-0022824,A.Lee,1,ARI,0,0,0.0,0,0.0,0.0,...,NaN,0,NaN,NaN,NaN,NaN,0.0,-2.00,-2.00,F
3,00-0022924,B.Roethlisberger,17,PIT,419,649,3955.0,24,10.0,40.0,...,NaN,0,NaN,NaN,NaN,NaN,0.0,234.60,234.60,F
4,00-0023459,A.Rodgers,17,GB,386,560,4340.0,37,4.0,35.0,...,-1.243446,0,0.000000,0.032258,-0.011299,0.040477,0.0,341.30,342.30,F
5,00-0023682,R.Fitzpatrick,1,WAS,3,6,13.0,0,0.0,1.0,...,NaN,0,NaN,NaN,NaN,NaN,0.0,0.72,0.72,F
6,00-0024243,M.Lewis,14,GB,0,0,0.0,0,0.0,0.0,...,3.289367,0,2.098039,0.062722,0.029130,0.114474,0.0,19.40,43.40,F
7,00-0024417,S.Koch,1,BAL,0,1,0.0,0,0.0,0.0,...,NaN,0,NaN,NaN,NaN,NaN,0.0,0.00,0.00,F
8,00-0025394,A.Peterson,4,SEA,0,0,0.0,0,0.0,0.0,...,-1.277645,0,-1.333333,0.038238,-0.016891,0.045534,0.0,22.60,26.60,F
9,00-0026035,D.Amendola,8,HOU,0,1,0.0,0,0.0,0.0,...,-0.118616,1,0.772586,0.148022,0.146186,0.324364,0.0,44.80,68.80,F
10,00-0026143,M.Ryan,17,ATL,375,560,3968.0,20,12.0,40.0,...,NaN,0,NaN,NaN,NaN,NaN,0.0,222.92,222.92,F


In [35]:
test = df_weekly_stats.iloc[0:17,47]
print(test)
st.variance(test)

1     27.16
2     29.64
3     28.68
4     11.06
5     37.74
6     17.98
7     24.44
8     25.20
9     13.00
10    19.28
11    11.24
12    28.62
13    30.12
14     4.76
15    14.38
16    26.40
17    25.04
Name: fantasy_points_ppr, dtype: float64


76.69931176470587

In [66]:
player_scores = {"" : []}
name = ""
for index, player_name in df_weekly_stats.iterrows():
    name = df_weekly_stats["player_name"][index]
    if(name not in player_scores.keys()):
        player_scores[name] = []
    player_scores[name].append(df_weekly_stats["fantasy_points_ppr"][index])

print(player_scores["T.Brady"])

[27.16, 29.64, 28.68, 11.06, 37.739999999999995, 17.980000000000004, 24.439999999999998, 25.2, 13.0, 19.28, 11.24, 28.619999999999997, 30.12, 4.760000000000001, 14.379999999999999, 26.4, 25.04]


In [99]:
player_var = {"" : 0}
for x in player_scores:
    if(len(player_scores[x]) > 1): 
        player_var[x] = st.variance(player_scores[x])

In [100]:
st.mean(player_var.values())
##st.variance(player_var.values())
player_grade = {"" : "F"}
for x in player_var:
    var = player_var[x]
    if(var < 40):
        player_grade[x] = "A"
    elif(var < 60):
        player_grade[x] = "B"
    elif(var < 90):
        player_grade[x] = "C"
    else:
        player_grade[x] = "D"
print(player_grade)


{'': 'A', 'T.Brady': 'C', 'B.Roethlisberger': 'A', 'Aa.Rodgers': 'C', 'A.Rodgers': 'B', 'M.Lewis': 'A', 'A.Peterson': 'A', 'D.Amendola': 'D', 'M.Ryan': 'C', 'J.Flacco': 'C', 'D.Jackson': 'B', 'C.Henne': 'A', 'J.Johnson': 'B', 'M.Stafford': 'B', 'B.Hoyer': 'A', 'J.Cook': 'A', 'R.Gronkowski': 'C', 'E.Sanders': 'B', 'C.McCoy': 'D', 'A.Roberts': 'A', 'J.Graham': 'A', 'A.Brown': 'D', 'T.Williams': 'A', 'C.Newton': 'D', 'A.Green': 'A', 'J.Jones': 'A', 'B.Gabbert': 'A', 'M.Ingram': 'A', 'A.Dalton': 'A', 'K.Rudolph': 'A', 'R.Cobb': 'B', 'L.Stocker': 'A', 'L.Smith': 'A', 'Ty.Taylor': 'A', 'T.Taylor': 'B', 'C.Hogan': 'A', 'J.Hekker': 'A', 'C.Keenum': 'B', 'C.Beasley': 'B', 'B.Bolden': 'A', 'R.Wilson': 'C', 'T.Benjamin': 'A', 'M.Jones': 'B', 'K.Cousins': 'A', 'T.Hilton': 'A', 'M.Sanu': 'A', 'J.Gordon': 'A', 'R.Tannehill': 'A', 'D.Fells': 'A', 'K.Juszczyk': 'A', 'A.Thielen': 'C', 'Z.Ertz': 'B', 'M.Goodwin': 'A', 'K.Stills': 'A', 'R.Griffin': 'A', 'D.Harris': 'B', 'J.Doyle': 'A', 'K.Allen': 'A', 'R

In [119]:
df_official_player_stats["consistency_grade"] = "F"

for index, player_name in df_official_player_stats.iterrows():
    pname = df_official_player_stats["player_name"][index]
    ##print(len(player_scores[pname]))
    if(player_scores.__contains__(pname)):
      if (len(player_scores[pname]) > 1):
        df_official_player_stats["consistency_grade"][index] = player_grade[pname]
df_official_player_stats.head(20)

/var/folders/bv/rzcr2l7j0fg9_28jldmf2kv40000gn/T/ipykernel_23778/1927082160.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_official_player_stats["consistency_grade"][index] = player_grade[pname]


,player_id,player_name,games,recent_team,completions,attempts,passing_yards,passing_tds,interceptions,sacks,...,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr,consistency_grade
1,00-0019596,T.Brady,19,TB,544,810,5916.0,46,13.0,29.0,...,NaN,0,NaN,NaN,NaN,NaN,0.0,406.74,406.74,C
2,00-0022824,A.Lee,1,ARI,0,0,0.0,0,0.0,0.0,...,NaN,0,NaN,NaN,NaN,NaN,0.0,-2.00,-2.00,F
3,00-0022924,B.Roethlisberger,17,PIT,419,649,3955.0,24,10.0,40.0,...,NaN,0,NaN,NaN,NaN,NaN,0.0,234.60,234.60,A
4,00-0023459,A.Rodgers,17,GB,386,560,4340.0,37,4.0,35.0,...,-1.243446,0,0.000000,0.032258,-0.011299,0.040477,0.0,341.30,342.30,B
5,00-0023682,R.Fitzpatrick,1,WAS,3,6,13.0,0,0.0,1.0,...,NaN,0,NaN,NaN,NaN,NaN,0.0,0.72,0.72,F
6,00-0024243,M.Lewis,14,GB,0,0,0.0,0,0.0,0.0,...,3.289367,0,2.098039,0.062722,0.029130,0.114474,0.0,19.40,43.40,A
7,00-0024417,S.Koch,1,BAL,0,1,0.0,0,0.0,0.0,...,NaN,0,NaN,NaN,NaN,NaN,0.0,0.00,0.00,F
8,00-0025394,A.Peterson,4,SEA,0,0,0.0,0,0.0,0.0,...,-1.277645,0,-1.333333,0.038238,-0.016891,0.045534,0.0,22.60,26.60,A
9,00-0026035,D.Amendola,8,HOU,0,1,0.0,0,0.0,0.0,...,-0.118616,1,0.772586,0.148022,0.146186,0.324364,0.0,44.80,68.80,D
10,00-0026143,M.Ryan,17,ATL,375,560,3968.0,20,12.0,40.0,...,NaN,0,NaN,NaN,NaN,NaN,0.0,222.92,222.92,C
